In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

### load dataset
data = pd.read_csv('churn_modelling_demo.csv')

### drop unwanted columns
data = data.drop(['CustomerID'], axis=1)

### encoding for categorical variable
label_gender_encode = LabelEncoder()
data['Gender'] = label_gender_encode.fit_transform(data['Gender'])

## for Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_geography_encoder=OneHotEncoder(sparse_output=False)
geo_encoder=onehot_geography_encoder.fit_transform(data[['Geography']])

geo_encoder_df = pd.DataFrame(geo_encoder, columns=onehot_geography_encoder.get_feature_names_out(['Geography']))

## Combine one hot encoder columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)
data.head()

### save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_gender_encode, file)
with open('onehot_geography_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_geography_encoder,file)

### divide the dataset into independent and dependent features

## Independent
X=data.drop('Exited', axis=1)
## Dependent
y = data['Exited']

## Split the data in training and testing sets;
X_train,X_test,Y_train,Y_test=train_test_split(X, y, test_size=0.2, random_state=42)

## Scale these features
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
    


AttributeError: 'DataFrame' object has no attribute 'train'

In [35]:
### Train the model on the scaler file
from numpy import histogram
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
import datetime

## Build our ANN model
##1. Make a sequential model && Create different layers
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1, activation='sigmoid') # Output layer
])

## Display model summary
## model.summary()

## compile the model and give accuracy 
model.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=['accuracy'])

### Setup tensorboard 
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

## Set up Early stopping if you dont' find consider changes in the loss value till 5 Epoch cycle
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

## Train the model now 
history=model.fit(
    X_train,Y_train,validation_data=(X_test, Y_test), epochs=1000,
    callbacks=[tensorflow_callback, early_stopping_callback]
)


e:\codes\ai\project_1\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.5047 - loss: 0.7059 - val_accuracy: 0.5090 - val_loss: 0.6931
Epoch 2/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5157 - loss: 0.6929 - val_accuracy: 0.5065 - val_loss: 0.6931
Epoch 3/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5015 - loss: 0.6924 - val_accuracy: 0.4890 - val_loss: 0.6930
Epoch 4/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5201 - loss: 0.6927 - val_accuracy: 0.4945 - val_loss: 0.6933
Epoch 5/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5101 - loss: 0.6931 - val_accuracy: 0.4940 - val_loss: 0.6935
Epoch 6/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5087 - loss: 0.6934 - val_accuracy: 0.4910 - val_loss: 0.6946
Epoch 7/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5068 - loss: 0.6936 - val_accuracy: 0.4870 - val_loss: 0.6952
Epoch 8/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5184 - loss: 0.6917 

In [36]:
## save the model file
model.save('model.h5')

In [ ]:
### load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --log_dir logs/fit20241026-114108/